# Interactively Writing Web Applications

Reimagining the world of web development with Klein and Jupyter

**Moshe Zadka**

**https://cobordism.com**

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## To The Editor...And Beyond?

* Interactive
* Development
* Environment

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Auto-Reloading: A Non-Solution to a Non-Problem

* Complicated and fragile
* Downtime
* Is it reloaded? But how about now?

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## What If I Told You?

I will write a web application during this talk.
At each step,
it will be usable.

There will be no reloading...just progress.

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Twisted, Meet Jupyter

* Klein on Twisted
* Twisted on Asyncioreactor
* Jupyter on Asyncio event loop

In [1]:
from twisted.internet import asyncioreactor
asyncioreactor.install()

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## The Imports

In Lisp, every file ends with `))))))))))`.

In Python, every file starts with `import import import`

In [2]:
from twisted.internet import endpoints
from twisted.web import server, template
from twisted.web.template import tags, slot
from twisted.internet import reactor, defer, task
from klein import app, resource, route, Klein, Plating, Form, Field, Requirer, SessionProcurer
from klein.interfaces import ISession
from klein.storage.memory import MemorySessionStore
import functools
import operator
import attr

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## When Endpoint Met Resource

* Klein wants to serve web requests, but doesn't know how to network
* Endpoints know how to network, but not how to send content

The romantic comedy coming soon to a conference near you!

In [3]:
endpoint = endpoints.serverFromString(reactor, "tcp:8000")
site = server.Site(resource())
endpoint.listen(site)

<Deferred at 0x7f84c7055490 current result: <<class 'twisted.internet.tcp.Port'> of <class 'twisted.web.server.Site'> on 8000>>

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Let's Check It Out

In [4]:
import webbrowser
webbrowser.open("http://localhost:8000")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## If You Don't Succeed At First, Redefine Success

We are not in the *web development business*.

We are in the *incremental web development business*.

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Our First Page

The root, using the classical CS tool "Hello world"

In [5]:
@route("/")
def welcome(request):
    return "<html>Hello world</html>"
webbrowser.open("http://localhost:8000")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## No Reloading -- No Race Conditions

In [6]:
@route('/about')
def about(request):
    return 'This is the about page'
@route("/")
def welcome(request):
    return '<html>Hello world. See <a href="/about">About</a></html>'
webbrowser.open("http://localhost:8000")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Modifying Routes

Added a link to an existing route -- and it changed!

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Web Applications Do Stuff

Static pages can be served with simpler techniques

In [38]:
@route('/double/<int:number>')
def double(request, number):
    result = number + number
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/double/12")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Imagine

* Real applications have backends
* Demo applications have...mock ends?
* Fictional world: Computation as a Service

In [8]:
delayed_multiplication = functools.partial(task.deferLater, reactor, 5, operator.mul)

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Using Slow Backends

All backends are slow...if your users are impatient enough

In [9]:
@route('/hyp_squared/<float:a>/<float:b>')
@defer.inlineCallbacks
def hyp_squared(request, a, b):
    a_squared = yield delayed_multiplication(a, a)
    b_squared = yield delayed_multiplication(b, b)
    result = a_squared + b_squared
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/hyp_squared/3.0/4.0")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Using Slow Backends

* Scaling the backend is someone else's problem

In [10]:
@route('/hyp_squared/<float:a>/<float:b>')
@defer.inlineCallbacks
def hyp_squared(request, a, b):
    d_a_squared = delayed_multiplication(a, a)
    d_b_squared = delayed_multiplication(b, b)
    a_squared, b_squared = yield defer.gatherResults([d_a_squared, d_b_squared])
    result = a_squared + b_squared
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/hyp_squared/3.0/4.0")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Computatation as a Service

* Now, with negation

In [11]:
delayed_negation = functools.partial(task.deferLater, reactor, 2, operator.neg)

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Euclidean Distances

* CaaS : Demos as Databases : Applications

In [12]:
@route('/distance_squared/<float:x1>/<float:y1>/<float:x2>/<float:y2>')
@defer.inlineCallbacks
def distance_squared(request, x1, y1, x2, y2):
    x, y = map(delayed_negation, (x2, y2))
    x.addCallback(operator.add, x1)
    x.addCallback(lambda t: delayed_multiplication(t, t))
    y.addCallback(operator.add, y1)
    y.addCallback(lambda t: delayed_multiplication(t, t))
    squares = yield defer.gatherResults([x, y])
    result = sum(squares)
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/distance_squared/2.0/1.0/5.0/5.0")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Is It Worth It?

Yes

In [13]:
@route('/slow_distance_squared/<float:x1>/<float:y1>/<float:x2>/<float:y2>')
@defer.inlineCallbacks
def slow_distance_squared(request, x1, y1, x2, y2):
    x = x1 - (yield delayed_negation(x2))
    y = y1 - (yield delayed_negation(y2))
    x_sq = yield delayed_multiplication(x, x)
    y_sq = yield delayed_multiplication(y, y)
    result = x_sq + y_sq
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/slow_distance_squared/2.0/1.0/5.0/5.0")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Nicer HTML

Now...with templates

In [14]:
from twisted.web import template
class Greeter(template.Element):
    loader = template.XMLString("""\
         <html xmlns:t="http://twistedmatrix.com/ns/twisted.web.template/0.1">
         <h1>Greeting</h1>
        </html>
        """)

@route('/greeting/<string:whom>')
def greeting(request, whom):
    return Greeter()
webbrowser.open("http://localhost:8000/greeting/pyconweb")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

In [15]:
@attr.s
class Greeter(template.Element):
    whom = attr.ib()
    loader = template.XMLString("""\
         <html xmlns:t="http://twistedmatrix.com/ns/twisted.web.template/0.1">
         <h1>Greeting</h1>
        </html>
        """)

@route('/greeting/<string:whom>')
def greeting(request, whom):
    return Greeter(whom)
webbrowser.open("http://localhost:8000/greeting/pyconweb")

True

In [16]:
Greeter.loader = template.XMLString("""\
         <html xmlns:t="http://twistedmatrix.com/ns/twisted.web.template/0.1">
         <h1>Greeting</h1>
         <span t:render="name">Default name</span>
        </html
        """)

SAXParseException: <unknown>:4:8: unclosed token

In [17]:
Greeter.loader = template.XMLString("""\
         <html xmlns:t="http://twistedmatrix.com/ns/twisted.web.template/0.1">
         <h1>Greeting</h1>
         <span t:render="name">Default name</span>
        </html>
        """)
webbrowser.open("http://localhost:8000/greeting/pyconweb")

True

An error occurred while rendering the response.
Traceback (most recent call last):
  File "/home/moshez/src/ppb-game/build/penguin/lib/python3.7/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/home/moshez/src/ppb-game/build/penguin/lib/python3.7/site-packages/klein/_resource.py", line 220, in process
    renderElement(request, r)
  File "/home/moshez/src/ppb-game/build/penguin/lib/python3.7/site-packages/twisted/web/template.py", line 550, in renderElement
    d = flatten(request, element, request.write)
  File "/home/moshez/src/ppb-game/build/penguin/lib/python3.7/site-packages/twisted/web/_flatten.py", line 401, in flatten
    _writeFlattenedData(state, write, result)
--- <exception caught here> ---
  File "/home/moshez/src/ppb-game/build/penguin/lib/python3.7/site-packages/twisted/web/_flatten.py", line 362, in _writeFlattenedData
    element = next(state)
  File "/home/moshez/src/ppb-game/build

In [18]:
Greeter.name = template.renderer(lambda self, request, tag: self.whom)
webbrowser.open("http://localhost:8000/greeting/pyconweb")

True

In [19]:
webbrowser.open("http://localhost:8000/greeting/<blink>pyconweb")

True

In [20]:
Greeter.loader = template.XMLString("""\
         <html xmlns:t="http://twistedmatrix.com/ns/twisted.web.template/0.1">
         <h1>Greeting</h1>
         <h2><span t:render="name">Default name</span></h2>
        Your name has <span t:render="count">NUMBER</span> letters!
        </html>
        """)

Greeter.count = template.renderer(lambda self, request, tag: str(len(self.whom)))
webbrowser.open("http://localhost:8000/greeting/pyconweb")

True

In [21]:
Greeter.loader = template.XMLString("""\
         <html xmlns:t="http://twistedmatrix.com/ns/twisted.web.template/0.1">
         <h1>Greeting</h1>
         <h2><span t:render="name">Default name</span></h2>
        Your name has <span t:render="count">NUMBER</span> letters!
        </html>
        """)

@defer.inlineCallbacks
def count(self, request, tag):
    base = len(self.whom)
    squared = yield delayed_multiplication(base, base)
    return str(squared)

Greeter.count = template.renderer(count)
webbrowser.open("http://localhost:8000/greeting/pyconweb")

True

In [22]:
Greeter.loader = template.XMLString("""\
         <html xmlns:t="http://twistedmatrix.com/ns/twisted.web.template/0.1">
         <h1>Greeting</h1>
         <h2><span t:render="name">Default name</span></h2>
        <p>Your name has the square root of <span t:render="count">NUMBER</span> letters!</p>
        If you put repeat it 10 times, it will have <span t:render="tenx">NUMBER</span> letters!
        </html>
        """)

@defer.inlineCallbacks
def tenx(self, request, tag):
    base = len(self.whom)
    result = yield delayed_multiplication(base, 10)
    return str(result)

Greeter.tenx = template.renderer(tenx)
webbrowser.open("http://localhost:8000/greeting/pyconweb")

True

In [24]:
sessions = MemorySessionStore()

requirer = Requirer()

@requirer.prerequisite([ISession])
def procurer(request):
    return SessionProcurer(sessions).procureSession(request)

In [25]:
style = Plating(tags=tags.html(tags.body(
    tags.h1("Form example"),
    tags.div(slot(Plating.CONTENT))))

In [35]:
@requirer.require(
    style.routed(
        app.route("/form", methods=["POST"]),
        tags.div(tags.p('The name is: ', slot("name")),
        tags.p("The square is: ", slot("square")))
    ),
    root=Field.number(minimum=1, maximum=10), name=Field.text(),
)
@defer.inlineCallbacks
def postHandler(root, name):
    square = yield delayed_multiplication(root, root)
    return dict(name=name, square=square)

In [36]:
@requirer.require(
    style.routed(
        app.route("/form", methods=["GET"]),

        tags.div(slot("form_contents"))
    ),
    form_contents=Form.rendererFor(postHandler, action=u"/form?post=yes")
)
def formRenderer(form_contents):
    return dict(form_contents=form_contents)

In [37]:
webbrowser.open("http://localhost:8000/form")

True